<a href="https://colab.research.google.com/github/ParthikB/Vohoo-PyTorch/blob/master/cats_and_dogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# PREPROCESSING

In [10]:
import os
import cv2
from tqdm import tqdm
import numpy as np

                                  # VARIABLES #
DATA_EXTRACTED = True
DATA_CREATED   = True
DATA_BALANCED  = True
CLIP           = 1680  #Clip every feature lenght at CLIP is data is not balanced.
#____________________________________________________________________________________#
                              # DOWNLOADING DATA #
PATH = '/content/drive/My Drive/Colab Notebooks/pyTorch/data/'
os.chdir(PATH)

os.environ['KAGGLE_USERNAME'] = "parthikb" # username from the json file 
os.environ['KAGGLE_KEY'] = "079b1e8e73bd390b39218acc15c82b09" # key from the json file
!kaggle datasets download -d chetankv/dogs-cats-images
#____________________________________________________________________________________#
                              # EXTRACTING DATA #
if not DATA_EXTRACTED:
  from zipfile import ZipFile

  file_name = 'dogs-cats-images.zip'

  with ZipFile(file_name, 'r') as zip:
    print('Initiating Extraction...')
    zip.extractall()
    print('Done!')
#____________________________________________________________________________________#
                                # MOVING DATA #
# PATH += 'dog vs cat/dataset'
# os.chdir(PATH)

# To move the dataset folders...

# import shutil
# shutil.move(PATH+'/training_set', '/content/drive/My Drive/Colab Notebooks/pyTorch/data/dog vs cat/')
# shutil.move(PATH+'/test_set', '/content/drive/My Drive/Colab Notebooks/pyTorch/data/dog vs cat/')
#____________________________________________________________________________________#
                           # CREATING TRAINING SET #
PATH = '/content/drive/My Drive/Colab Notebooks/pyTorch/data/dog vs cat/training_set'
os.chdir(PATH)

class DogsVSCats:
  IMG_SIZE     = 50
  CATS         = os.path.join(PATH, 'cats') 
  DOGS         = os.path.join(PATH, 'dogs')
  labels       = {CATS:0, DOGS:1}
  training_data = []
  cat_count, dog_count = 0, 0

  def createTrainingData(self):
    try:
      for label in self.labels:
        os.chdir(label)
        data = os.listdir()
        for image in tqdm(data):
          img = os.path.join(label, image)
          img = cv2.imread(img, 0)

          one_hot = np.eye(len(self.labels))[self.labels[label]]
          img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
          
          self.training_data.append([np.array(img), one_hot])
          
          if label == self.CATS:
            self.cat_count += 1
          else:
            self.dog_count += 1
    except Exception as e:
      print(e)

    np.random.shuffle(self.training_data)
    np.save('/content/drive/My Drive/Colab Notebooks/pyTorch/data/dog vs cat/training_data.npy', self.training_data)
    print('Cats : ', self.cat_count)
    print('Dogs : ', self.dog_count)
    
if not DATA_CREATED:
  print('Creating Training set...')
  dogsVScats = DogsVSCats()
  dogsVScats.createTrainingData()

  training_data = np.load('/content/drive/My Drive/Colab Notebooks/pyTorch/data/dog vs cat/training_data.npy', allow_pickle=True)
len(training_data)
#____________________________________________________________________________________#
                              # BALANCING TRAINING SET #
if not DATA_BALANCED:
  print('Balancing Training set...')
  cat_counter, dog_counter = 0, 0
  balanced_training_data = []

  for data in tqdm(training_data):
    if data[1][0] == 1 and cat_counter < CLIP: #if cat
      cat_counter += 1
      balanced_training_data.append(data)
    elif data[1][1] == 1 and dog_counter < CLIP: #if dog
      dog_counter += 1
      balanced_training_data.append(data)

  training_data = balanced_training_data
#____________________________________________________________________________________#

print("Length of Final Training Set :", len(training_data))

dogs-cats-images.zip: Skipping, found more recently modified local copy (use --force to force download)
Length of Final Training Set : 5680


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class Net(nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
    self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5)
    self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5)

    self.flatten = None
    x = torch.randn(50, 50).view(-1, 1, 50, 50)
    x = self.convs(x)

    self.fc1   = nn.Linear(self.flatten, 512)
    self.fc2   = nn.Linear(512, 2)

    # x = self.forward(x)

  def convs(self, x):
    x = F.max_pool2d(F.relu(self.conv1(x)), kernel_size=(2, 2))
    x = F.max_pool2d(F.relu(self.conv2(x)), kernel_size=(2, 2))
    x = F.max_pool2d(F.relu(self.conv3(x)), kernel_size=(2, 2))
    print(x.shape)
    if not self.flatten:
      self.flatten = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
    return x


  def forward(self, x):
    x = self.convs(x)
    x = x.view(-1, self.flatten)
    x = F.relu(self.fc1(x))
    x = F.softmax(self.fc2(x), dim=1)
    return x


In [32]:
net = Net()

torch.Size([1, 128, 2, 2])
